In [1]:
import sys
sys.path.insert(0, '../')
from models.custom import AHOY

ahoy_ir = AHOY("../weights/yolov5n_IR.pt", "../weights/yolov5h_IR.pt", fp16=True)
ahoy_rgb = AHOY("../weights/yolov5n6.pt", "../weights/yolov5h6.pt", fp16=True)

🚀 SEA.AI custom version


YOLOv5 🚀 v7.0-336-gceaf2e3e Python-3.10.11 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11004MiB)

Fusing layers... 
Model summary: 157 layers, 1771342 parameters, 0 gradients, 4.2 GFLOPs
Loaded weights from ../weights/yolov5n_IR.pt
YOLOv5 🚀 v7.0-336-gceaf2e3e Python-3.10.11 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11004MiB)

Fusing layers... 
Model summary: 93 layers, 2818424 parameters, 0 gradients, 4.2 GFLOPs
Loaded weights from ../weights/yolov5h_IR.pt
YOLOv5 🚀 v7.0-336-gceaf2e3e Python-3.10.11 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11004MiB)

YOLOv5 🚀 v7.0-336-gceaf2e3e Python-3.10.11 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11004MiB)

Fusing layers... 
Model summary: 206 layers, 3102712 parameters, 0 gradients, 4.3 GFLOPs
Loaded weights from ../weights/yolov5n6.pt
YOLOv5 🚀 v7.0-336-gceaf2e3e Python-3.10.11 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11004MiB)

Fusing layers... 
Model summary: 108 layers, 3353720 parameters, 0 gradients, 4.6 GFLOPs
Lo

In [2]:
from models.custom import DAN
dan = DAN(model_a=ahoy_rgb, model_b=ahoy_ir)

In [3]:
from urllib.request import urlopen
import numpy as np
from scipy.optimize import curve_fit
from PIL import Image
import torch

from utils.general import non_max_suppression
from utils.horizon import draw_horizon, draw_bboxes
from inference.preprocessing import letterbox_image

url_ir = "https://github.com/SEA-AI/.github/blob/main/assets/example_1_IR.jpg?raw=true"
image_ir = np.array(Image.open(urlopen(url_ir)).convert("RGB"))
imgsz_ir = 640

url_rgb = "https://github.com/SEA-AI/.github/blob/main/assets/example_1_RGB.jpg?raw=true"
image_rgb = np.array(Image.open(urlopen(url_rgb)).convert("RGB"))
imgsz_rgb = 1280

def postprocess_x_pitch_theta(x_pitch: torch.Tensor, x_theta: torch.Tensor):
    # curve fitting
    def gaussian(x, A, mu, sigma):
        return A * np.exp(-(x - mu)**2 / (2 * sigma**2))

    x_pitch = x_pitch.softmax(-1).squeeze().cpu().numpy()
    x_theta = x_theta.softmax(-1).squeeze().cpu().numpy()

    # Initial guess for the parameters (amplitude, mean, standard deviation)
    pitch0 = [x_pitch.max(), x_pitch.argmax() / x_pitch.shape[-1], 0.001]
    theta0 = [x_theta.max(), x_theta.argmax() / x_theta.shape[-1], 0.001]

    x_data = np.linspace(0, 1, x_pitch.shape[0], endpoint=False)
    params_pitch, cov_pitch = curve_fit(gaussian, x_data, x_pitch, p0=pitch0)
    params_theta, cov_theta = curve_fit(gaussian, x_data, x_theta, p0=theta0)

    return params_pitch[1], params_theta[1]

def postprocess_dets(
        dets,
        conf_thres=0.25,
        iou_thres=0.1,
        classes=None,
        agnostic_nms=True,
        max_det=100
):           
    dets = non_max_suppression(dets, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
    return dets[0].cpu().numpy()

def preprocess(image, imgsz):
    x = letterbox_image(image, desired_size=(imgsz, imgsz))
    x = torch.from_numpy(x).permute(2,0,1).unsqueeze(0)
    x = x.to(ahoy_rgb.device)
    x = x.half() if ahoy_rgb.fp16 else x.float()
    x /= 255.0
    return x

with torch.no_grad():
    x_rgb = preprocess(image_rgb, imgsz_rgb)
    x_ir = preprocess(image_ir, imgsz_ir)

    (dets_rgb, x_pitch_rgb, x_theta_rgb), (dets_ir, x_pitch_ir, x_theta_ir) = dan(x_rgb, x_ir)

    dets_rgb = postprocess_dets(dets_rgb)
    dets_ir = postprocess_dets(dets_ir)

    pitch_rgb, theta_rgb = postprocess_x_pitch_theta(x_pitch_rgb, x_theta_rgb)
    pitch_ir, theta_ir = postprocess_x_pitch_theta(x_pitch_ir, x_theta_ir)

In [4]:
I = letterbox_image(image_ir, desired_size=(imgsz_ir, imgsz_ir))
diameter = 1
I = draw_bboxes(I, dets_ir, color=(0,0,255), thickness=1)
# I = draw_horizon(I, pitch_theta=(target[0][0], target[0][1]), color=(0,255,0), diameter=diameter)
I = draw_horizon(I, pitch_theta=(pitch_ir, theta_ir), color=(0,0,0), diameter=diameter)
I = Image.fromarray(I)
I #.reduce(2) if I.width >= 1024 else I

In [5]:
I = letterbox_image(image_rgb, desired_size=(imgsz_rgb, imgsz_rgb))
diameter = 1
I = draw_bboxes(I, dets_rgb, color=(0,0,255), thickness=1)
# I = draw_horizon(I, pitch_theta=(target[0][0], target[0][1]), color=(0,255,0), diameter=diameter)
I = draw_horizon(I, pitch_theta=(pitch_rgb, theta_rgb), color=(0,0,0), diameter=diameter)
I = Image.fromarray(I)
I #.reduce(2) if I.width >= 1024 else I

In [6]:
from pathlib import Path
from export import export_engine
from models.yolo import Detect

inplace = False # default=False
dynamic = False # default=False
batch_size = 2
name = f"dan_b{batch_size}"

# Update model
dan.eval()
for k, m in dan.named_modules():
    if isinstance(m, Detect):
        m.inplace = inplace
        m.dynamic = dynamic
        m.export = True

img_ir = torch.zeros((batch_size, 3, imgsz_ir, imgsz_ir), device=dan.device).byte()
img_rgb = torch.zeros((batch_size, 3, imgsz_rgb, imgsz_rgb), device=dan.device).byte()
images = (img_rgb, img_ir)

dan.register_io_hooks()
outs = dan(*images)
print([x.dtype for x in images], [o[0].dtype for out in outs for o in out])

export_engine(
    dan,
    im=images,
    file=Path(name),
    half=dan.fp16,
    #opset=12,
    dynamic=dynamic,
    simplify=False,
)


ONNX: starting export with onnx 1.13.1...


[torch.uint8, torch.uint8] [torch.float32, torch.float32, torch.float32, torch.float32, torch.float32, torch.float32]


/home/kevinserrano/GitHub/yolov5/hybrids/../models/yolo.py:153: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if profile:
/home/kevinserrano/GitHub/yolov5/hybrids/../models/yolo.py:157: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if visualize:
/home/kevinserrano/GitHub/yolov5/hybrids/../models/yolo.py:157: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize 

[03/26/2024-16:26:18] [TRT] [I] [MemUsageChange] Init CUDA: CPU +2, GPU +0, now: CPU 654, GPU 1448 (MiB)
[03/26/2024-16:26:22] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +895, GPU +172, now: CPU 1625, GPU 1620 (MiB)
[03/26/2024-16:26:22] [TRT] [I] ----------------------------------------------------------------
[03/26/2024-16:26:22] [TRT] [I] Input filename:   dan_b2.onnx
[03/26/2024-16:26:22] [TRT] [I] ONNX IR version:  0.0.7
[03/26/2024-16:26:22] [TRT] [I] Opset version:    12
[03/26/2024-16:26:22] [TRT] [I] Producer name:    pytorch
[03/26/2024-16:26:22] [TRT] [I] Producer version: 2.1.0
[03/26/2024-16:26:22] [TRT] [I] Domain:           
[03/26/2024-16:26:22] [TRT] [I] Model version:    0
[03/26/2024-16:26:22] [TRT] [I] Doc string:       
[03/26/2024-16:26:22] [TRT] [I] ----------------------------------------------------------------
[03/26/2024-16:26:22] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not